# KNN Model

Hello!

In [65]:
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install seaborn
!{sys.executable} -m pip install scikit-learn


[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 23.2.1 -> 25.1.1
[notice] To update, run: python3.11 -m pip install --upgrade pip


In [66]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [67]:
# Simply import the new dataset created from the EDA
df = pd.read_csv('/Users/blancavaldes/Desktop/PcPartPicker3000/assignment/df_engineered.csv')

# Display the first few rows of the dataframe
print("First 5 rows of the dataframe:")
print(df.head())

# Display a concise summary of the dataframe, including data types and non-null counts
print("\nDataFrame Info:")
df.info()

# Display basic descriptive statistics for numerical columns
print("\nDescriptive Statistics:")
print(df.describe())

First 5 rows of the dataframe:
                             titulo     tipo_de_producto              serie  \
0     Apple MacBook Air 13" 2024 M3  Portátil multimedia  Apple MacBook Air   
1     Apple MacBook Air 15" 2024 M3  Portátil multimedia  Apple MacBook Air   
2   Apple MacBook Pro 14" M4 (2024)  Portátil multimedia  Apple MacBook Pro   
3  ASUS TUF Gaming A15 FA506 (2024)      Portátil gaming    ASUS TUF Gaming   
4         GigaByte Aero 16 OLED BKF      Portátil gaming      Gigabyte Aero   

  pantalla_tecnologia            procesador disco_duro_tipo_de_disco_duro  \
0              Retina              Apple M3                           SSD   
1              Retina              Apple M3                           SSD   
2              Retina               Unknown                           SSD   
3             Full HD               Unknown                           SSD   
4                UHD+  Intel Core i7-13700H                           SSD   

  grafica_salida_de_video conec

## Unsupervised KNN

In [68]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.neighbors import NearestNeighbors
import pandas as pd # ensure pandas is imported if you haven't already in a previous cell

# Assuming 'df' is your DataFrame loaded in a previous cell:
# df = pd.read_csv('df_engineered.csv')

# Define the features to use and the identifier column
identifier_col = 'titulo'
features_to_use = [
    'tipo_de_producto',
    'serie',
    'procesador',
    'disco_duro_tipo_de_disco_duro',
    'grafica_tarjeta',
    'ram_memoria_gb',
    'disco_duro_capacidad_de_memoria_ssd_gb',
    'custom_category',
    'pantalla_tamano_pulgadas'
]

# Create a new DataFrame with only the selected features and the identifier
df_selected = df[[identifier_col] + features_to_use].copy()

print("Selected DataFrame head:")
print(df_selected.head())
print(f"\nShape of selected DataFrame: {df_selected.shape}")

Selected DataFrame head:
                             titulo     tipo_de_producto              serie  \
0     Apple MacBook Air 13" 2024 M3  Portátil multimedia  Apple MacBook Air   
1     Apple MacBook Air 15" 2024 M3  Portátil multimedia  Apple MacBook Air   
2   Apple MacBook Pro 14" M4 (2024)  Portátil multimedia  Apple MacBook Pro   
3  ASUS TUF Gaming A15 FA506 (2024)      Portátil gaming    ASUS TUF Gaming   
4         GigaByte Aero 16 OLED BKF      Portátil gaming      Gigabyte Aero   

             procesador disco_duro_tipo_de_disco_duro         grafica_tarjeta  \
0              Apple M3                           SSD  Intel Iris Xe Graphics   
1              Apple M3                           SSD  Intel Iris Xe Graphics   
2               Unknown                           SSD  Intel Iris Xe Graphics   
3               Unknown                           SSD  Intel Iris Xe Graphics   
4  Intel Core i7-13700H                           SSD  Intel Iris Xe Graphics   

   ram_memori

In [69]:
# CELL 2 (from previous instructions)
# Identify numerical and categorical columns from the selected features
numerical_features = df_selected[features_to_use].select_dtypes(include=np.number).columns.tolist()
categorical_features = df_selected[features_to_use].select_dtypes(include='object').columns.tolist()

print(f"\nNumerical features: {numerical_features}")
print(f"Categorical features: {categorical_features}")


Numerical features: ['ram_memoria_gb', 'disco_duro_capacidad_de_memoria_ssd_gb', 'pantalla_tamano_pulgadas']
Categorical features: ['tipo_de_producto', 'serie', 'procesador', 'disco_duro_tipo_de_disco_duro', 'grafica_tarjeta', 'custom_category']


In [70]:
# Preprocessing for numerical data:
# 1. Impute missing values with the median
# 2. Scale data using StandardScaler
numerical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocessing for categorical data:
# 1. Impute missing values with a constant string 'missing'
# 2. Apply OneHotEncoder (handle_unknown='ignore' will prevent errors if new categories appear in future data)
categorical_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)) # sparse_output=False for dense array
])

# Create a ColumnTransformer to apply different transformations to different columns
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_pipeline, numerical_features),
        ('cat', categorical_pipeline, categorical_features)
    ],
    remainder='passthrough' # Keep other columns (if any, though we selected all)
)

In [71]:
# Apply the transformations
# We fit_transform on the features, excluding the identifier column for now
X_processed_array = preprocessor.fit_transform(df_selected[features_to_use])

# Get feature names after one-hot encoding for the new DataFrame
# This can be a bit tricky with ColumnTransformer, so we'll construct them
# For one-hot encoded columns:
ohe_feature_names = preprocessor.named_transformers_['cat'].named_steps['onehot'].get_feature_names_out(categorical_features)

# Combine with numerical feature names
all_feature_names = numerical_features + ohe_feature_names.tolist()

# Create a new DataFrame with the processed features
X_processed = pd.DataFrame(X_processed_array, columns=all_feature_names, index=df_selected.index)

print("\nProcessed features DataFrame head:")
print(X_processed.head())
print(f"\nShape of processed features DataFrame: {X_processed.shape}")

# Keep the titles for later reference
product_titles = df_selected[identifier_col]


Processed features DataFrame head:
   ram_memoria_gb  disco_duro_capacidad_de_memoria_ssd_gb  \
0       -0.284095                               -0.544318   
1       -0.284095                               -0.544318   
2       -0.284095                               -0.544318   
3       -0.284095                               -0.544318   
4       -0.284095                                0.533246   

   pantalla_tamano_pulgadas  tipo_de_producto_Barebone  \
0                 -2.130644                        0.0   
1                 -0.124706                        0.0   
2                 -1.422666                        0.0   
3                  0.229283                        0.0   
4                  0.701269                        0.0   

   tipo_de_producto_Chromebook  tipo_de_producto_Kit ampliación PC  \
0                          0.0                                 0.0   
1                          0.0                                 0.0   
2                          0.0        

In [72]:
from sklearn.neighbors import NearestNeighbors

# Define the number of neighbors to find (k)
# We'll ask for k+1 because the closest neighbor to an item is usually the item itself
n_neighbors_to_find = 6 # This will find the item itself + 5 most similar items

# Initialize the NearestNeighbors model
# 'metric' can be 'euclidean', 'manhattan', 'cosine', etc.
# 'cosine' is often good for high-dimensional data like one-hot encoded text features
# 'minkowski' with p=2 is equivalent to 'euclidean'
knn_model = NearestNeighbors(n_neighbors=n_neighbors_to_find, metric='cosine', algorithm='brute')

# Fit the model to your processed data
knn_model.fit(X_processed)

print(f"NearestNeighbors model fitted with k={n_neighbors_to_find} and metric='cosine'")

NearestNeighbors model fitted with k=6 and metric='cosine'


In [80]:
# CELL 6 (REVISED) - Function to Get Similar Products with Prices

def get_similar_products_with_prices(product_title_query, k=5):
    """
    Finds k most similar products to the given product_title,
    and includes their feature distance, real price, and predicted price.

    Args:
        product_title_query (str): The title of the product to find neighbors for.
        k (int): The number of similar products to return.

    Returns:
        pandas.DataFrame: A DataFrame containing the titles, feature distances,
                          real prices, and predicted prices of the k most similar products.
                          Returns None if the product_title_query is not found.
    """
    # Ensure product_titles Series, knn_model (for similarity),
    # X_processed, df_selected (for real price), and price_regressor_final are accessible.

    if product_title_query not in product_titles.values:
        print(f"Query Product '{product_title_query}' not found in the dataset.")
        return None

    # --- For the Query Product ---
    query_product_idx = product_titles[product_titles == product_title_query].index[0]
    query_product_features = X_processed.loc[query_product_idx].values.reshape(1, -1)
    
    query_real_price = None
    query_predicted_price = None

    if query_product_idx in X_price_features.index: # Check if price data was available for this product
        query_real_price = y_price.loc[query_product_idx] # Get from our cleaned y_price
        query_product_price_features = X_price_features.loc[query_product_idx].values.reshape(1,-1)
        query_predicted_price = price_regressor_final.predict(query_product_price_features)[0]
    else:
        # If no price data for query product, we can still try to predict if it has features
        # This case might occur if original price was NaN
        # To predict, it must have been part of X_processed
        if query_product_idx in X_processed.index:
             # We can attempt to predict price if its features are in X_processed
             # (which means it went through the main preprocessor)
             try:
                query_predicted_price = price_regressor_final.predict(query_product_features)[0]
             except Exception as e:
                print(f"Could not predict price for query item {product_title_query}: {e}")
                query_predicted_price = np.nan # Or some other placeholder


    print(f"\n--- Query Product: {product_title_query} ---")
    print(f"Real Price: {query_real_price if query_real_price is not None else 'N/A (original price was NaN or not in price model training set)'}")
    print(f"Predicted Price: {query_predicted_price:.2f}" if query_predicted_price is not None and not np.isnan(query_predicted_price) else "N/A (could not predict)")
    print("------------------------------------")


    # --- Find Similar Products based on Features ---
    distances, indices = knn_model.kneighbors(query_product_features, n_neighbors=k + 1)

    # Exclude the query product itself from the list of similar items
    similar_product_indices = indices[0][1:]
    similar_product_feature_distances = distances[0][1:]

    results_data = []
    for i, idx in enumerate(similar_product_indices):
        title = product_titles.iloc[idx]
        feature_dist = similar_product_feature_distances[i]
        
        real_p = None
        predicted_p = None

        # Check if this similar product had its price data used for the price regressor
        if idx in X_price_features.index: # Ensure index 'idx' exists in X_price_features
            real_p = y_price.loc[idx] # Get from our cleaned y_price
            # Get its specific features used for price prediction
            item_price_features = X_price_features.loc[idx].values.reshape(1,-1)
            predicted_p = price_regressor_final.predict(item_price_features)[0]
        else:
            # If original price was NaN, we can still try to predict if it has features
            # To predict, it must have been part of X_processed
            if idx in X_processed.index:
                item_features_for_pred = X_processed.loc[idx].values.reshape(1,-1)
                try:
                    predicted_p = price_regressor_final.predict(item_features_for_pred)[0]
                except Exception as e:
                    print(f"Could not predict price for item {title}: {e}")
                    predicted_p = np.nan


        results_data.append({
            'Similar Product Title': title,
            'Feature Similarity Distance': feature_dist,
            'Real Price': real_p if real_p is not None else np.nan,
            'Predicted Price': predicted_p if predicted_p is not None and not np.isnan(predicted_p) else np.nan
        })

    return pd.DataFrame(results_data)

print("Function 'get_similar_products_with_prices' is defined.")

Function 'get_similar_products_with_prices' is defined.


In [84]:
# CELL 7 (REVISED) - Test the Enhanced Recommendation Function

# Ensure product_titles and other necessary variables are loaded from previous cells.

print("="*50)
print("Testing KNN Recommendations with Price Info")
print("="*50)

# --- Test Case 1: First product in the dataset ---
# (Make sure this product was likely included in price model training, i.e., its price wasn't NaN)
test_title_1 = product_titles.iloc[0] # Or pick one you know has a price
if test_title_1 in product_titles.values:
    print(f"\n--- Finding products similar to: '{test_title_1}' (k=5) ---")
    # Use the new function name
    similar_items_1 = get_similar_products_with_prices(test_title_1, k=5)
    if similar_items_1 is not None:
        print(similar_items_1.to_string()) # .to_string() for better display if wide
else:
    print(f"Test product '{test_title_1}' not found.")


# --- Test Case 2: Another product (e.g., from middle of dataset) ---
if len(product_titles) > 100:
    # Try to pick an index that likely exists in y_price after dropna
    # This is a heuristic; ideally, pick a title you know has a non-NaN price
    potential_idx = len(product_titles) // 2
    if potential_idx in product_titles.index and product_titles.loc[potential_idx] in product_titles.values:
         test_title_2 = product_titles.loc[potential_idx]
         if df_selected.loc[df_selected[identifier_col] == test_title_2, target_price_column].notna().any(): # Check if original price is not NaN
            print(f"\n--- Finding products similar to: '{test_title_2}' (k=5) ---")
            similar_items_2 = get_similar_products_with_prices(test_title_2, k=5)
            if similar_items_2 is not None:
                print(similar_items_2.to_string())
         else:
            print(f"\nSkipping test_title_2 ('{test_title_2}') as its original price might be NaN.")
    else:
        print("\nCould not find a suitable test_title_2.")
else:
    print("\nDataset too small for Test Case 2 with this logic.")


# Add more test cases as needed, particularly for items you are interested in or edge cases.

print("\n" + "="*50)
print("Review the results. Consider:")
print("1. Feature Similarity Distance: Lower is more similar based on specs.")
print("2. Real Price: Actual price of the similar items.")
print("3. Predicted Price: Price predicted by the KNeighborsRegressor for those items.")
print("4. How well do the predicted prices align with real prices for these similar items?")
print("5. The evaluation metrics for the price regressor (MAE, R²) give a general idea of its accuracy.")
print("="*50)

Testing KNN Recommendations with Price Info

--- Finding products similar to: 'Apple MacBook Air 13" 2024 M3' (k=5) ---

--- Query Product: Apple MacBook Air 13" 2024 M3 ---
Real Price: 1656.85
Predicted Price: 1390.75
------------------------------------
                               Similar Product Title  Feature Similarity Distance  Real Price  Predicted Price
0  Apple MacBook Air 13" 2024 M3\nStarlight MXCU3Y/A                     0.000000     1424.93         1390.750
1                      Apple MacBook Air 13" 2024 M3                     0.000000     1656.85         1390.750
2                         Apple MacBook Air 13" 2020                     0.088354     1269.67         1250.462
3                      Apple MacBook Air 13" 2020 M1                     0.088354     1247.06         1250.462
4                      Apple MacBook Air 13" 2022 M2                     0.091603     1304.34         1257.088

--- Finding products similar to: 'Beelink SER6 Pro (7735HS 32 + 500G)' (k=5) 

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but NearestNeighbors was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.

                                         Similar Product Title  Feature Similarity Distance  Real Price  Predicted Price
0                   Minisforum UM773 Lite (32GB RAM / 1TB SSD)                     0.081206      463.99          551.916
1                            Beelink EQR6\n6900HX 32GB + 500GB                     0.132878      569.00          524.794
2                      Minisforum UM69003EU (32GB + 512GB SSD)                     0.133185      469.99          524.794
3                    Acemagician AM08 Pro\n6900HX 32GB + 512GB                     0.133185      589.00          524.794
4  Minisforum UM690 Pro (Ryzen 9 6900HX / 32GB DDR5 / 1TB SSD)                     0.214453      495.99          527.994

Review the results. Consider:
1. Feature Similarity Distance: Lower is more similar based on specs.
2. Real Price: Actual price of the similar items.
3. Predicted Price: Price predicted by the KNeighborsRegressor for those items.
4. How well do the predicted prices align w

## Regressor

In [91]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np # ensure numpy is imported

# --- Define the target price column ---
target_price_column = 'precio_mean'

# Ensure this column is in df_selected and handle missing values if any in the target
if target_price_column not in df_selected.columns:
    # If not already selected, add it from the original df
    if target_price_column in df.columns:
        df_selected[target_price_column] = df[target_price_column]
    else:
        raise ValueError(f"Target price column '{target_price_column}' not found in the original DataFrame.")

# Create the target variable y_price
# Drop rows where the target price is NaN, and also drop corresponding rows in X_processed
y_price_full = df_selected[target_price_column].copy()
valid_price_indices = y_price_full.dropna().index

X_price_features = X_processed.loc[valid_price_indices]
y_price = y_price_full.loc[valid_price_indices]

print(f"Shape of features for price prediction (X_price_features): {X_price_features.shape}")
print(f"Shape of target price variable (y_price): {y_price.shape}")

# Split data for price prediction model evaluation
X_train_price, X_test_price, y_train_price, y_test_price = train_test_split(
    X_price_features, y_price, test_size=0.2, random_state=42
)

print(f"X_train_price shape: {X_train_price.shape}, y_train_price shape: {y_train_price.shape}")
print(f"X_test_price shape: {X_test_price.shape}, y_test_price shape: {y_test_price.shape}")

Shape of features for price prediction (X_price_features): (7914, 860)
Shape of target price variable (y_price): (7914,)
X_train_price shape: (6331, 860), y_train_price shape: (6331,)
X_test_price shape: (1583, 860), y_test_price shape: (1583,)


In [92]:
# Initialize KNeighborsRegressor
# You can experiment with n_neighbors for the regressor
price_regressor = KNeighborsRegressor(n_neighbors=5, metric='cosine') # Using cosine like our similarity model

# Train the price regressor
price_regressor.fit(X_train_price, y_train_price)
print("Price prediction model (KNeighborsRegressor) trained.")

# Make predictions on the test set
y_pred_price = price_regressor.predict(X_test_price)

# Evaluate the price prediction model
mae = mean_absolute_error(y_test_price, y_pred_price)
mse = mean_squared_error(y_test_price, y_pred_price)
rmse = np.sqrt(mse)
r2 = r2_score(y_test_price, y_pred_price)

print("\n--- Price Prediction Model Evaluation (on test set) ---")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"R-squared (R²): {r2:.2f}")
print("---------------------------------------------------------")

# Now, train the price regressor on the FULL available dataset (X_price_features, y_price)
# for making predictions on similar items later.
price_regressor_final = KNeighborsRegressor(n_neighbors=5, metric='cosine')
price_regressor_final.fit(X_price_features, y_price)
print("\nFinal price prediction model trained on all available price data.")

Price prediction model (KNeighborsRegressor) trained.

--- Price Prediction Model Evaluation (on test set) ---
Mean Absolute Error (MAE): 396.73
Mean Squared Error (MSE): 532606.27
Root Mean Squared Error (RMSE): 729.80
R-squared (R²): 0.59
---------------------------------------------------------

Final price prediction model trained on all available price data.
